## Fetch the runs and collate information in a dataframe

In [ ]:
import wandb

api = wandb.Api()
entity, project = "sayakpaul", "keras-xla-benchmarks"  
runs = api.runs(entity + "/" + project) 
print(f"Total runs: {len(runs)}")

In [ ]:
from model_mapping import MODEL_NAME_MAPPING

all_variants = [
    variant for k in MODEL_NAME_MAPPING for variant in MODEL_NAME_MAPPING[k]
]

In [ ]:
import pandas as pd

resolutions = []
accelerators = []

model_families = []
model_variants = []
xla_status = []

flops = []
params = []
throughputs = []

for run in runs:
    run_config = run.config
    run_summary = run.summary._json_dict

    if run_config["variant"] in all_variants:
        model_families.append(run_config["family"])
        model_variants.append(run_config["variant"])
        resolutions.append(run_config["resolution"])
        xla_status.append(run_config["xla"])

        accelerator_name = run.name.split("@")[-1].split("-")[1]
        accelerators.append(accelerator_name)

        flops.append(run_summary["FLOPs (giga)"])
        params.append(run_summary["Num parameters (million)"])
        throughputs.append(run_summary["Throughput (samples/sec)"])

viz_df = pd.DataFrame(
    {
        "model_family": model_families,
        "model_variant": model_variants,
        "resolution": resolutions,
        "xla": xla_status,
        "accelerator": accelerators,
        "flop (giga)": flops,
        "params (million)": params,
        "throughput (samples/sec)": throughputs,
    }
)
viz_df.head()

In [ ]:
viz_df["accelerator"].unique()

## Filter w.r.t accelerator

In [ ]:
def plot_topk_per_accelerator(
    accelerator="a100", topk=10, resolution=224, xla_status=True
):
    filtered_df = viz_df[viz_df["accelerator"] == accelerator]
    subset_df = filtered_df.query(f"resolution == {resolution} and xla == {xla_status}")
    topk_df = subset_df.nlargest(topk, ["throughput (samples/sec)"])
    return topk_df

In [ ]:
# Adapted from
# https://github.com/nlp-with-transformers/notebooks/blob/main/08_model-compression.ipynb

import matplotlib.pyplot as plt


def plot_metrics(df, savefig=False):
    for model_variant in df["model_variant"]:
        filtered = df.query(f"model_variant == '{model_variant}'")
        plt.scatter(
            filtered["flop (giga)"],
            filtered["throughput (samples/sec)"],
            alpha=0.5,
            s=filtered["params (million)"] * 5,
            label=model_variant,
            marker="o",
        )

    legend = plt.legend(bbox_to_anchor=(1, 1))
    for handle in legend.legendHandles:
        handle.set_sizes([20])

    plt.ylabel("Throughput (samples/sec)", fontsize=14)
    plt.xlabel("FLOPS (giga)", fontsize=14)

    accelerator_name = df["accelerator"].unique()[0]
    resolution = df["resolution"].unique()[0]
    xla_status = df["xla"].unique()[0]
    plt.title(
        f"Accelerator: {accelerator_name}, Resolution: {resolution}, XLA: {xla_status}",
        fontsize=14,
    )
    if not savefig:
        plt.show()
    else:
        plot_name = f"{accelerator_name}_{resolution}_{xla_status}.png"
        plt.savefig(plot_name, dpi=300, bbox_inches="tight")

### A100

In [ ]:
a100_df = plot_topk_per_accelerator("a100")
plot_metrics(a100_df, True)

### V100

In [ ]:
v100_df = plot_topk_per_accelerator("v100")
plot_metrics(v100_df, True)

### T4

In [ ]:
t4_df = plot_topk_per_accelerator("t4")
plot_metrics(t4_df, True)

For each accelerator type, the trend of the models leading to the highest amount of throughput seem to vary a little bit. 

## Resolution-wise throughput distribution

In [ ]:
viz_df.resolution.unique()

In [ ]:
# Grouping the dataframe by unique resolutions and finding the 
# model variant with highest throughput per group.
grouped = viz_df.groupby("resolution")["throughput (samples/sec)"].idxmax()

# Selecting the rows with the highest throughput per group.
result = viz_df.loc[grouped, viz_df.columns]
result

This means that for each resolution the highest throughput seems to have a relationship with the accelerator used for running the benchmark. 

What happens if we take a group by the accelerator too?

In [ ]:
grouped = viz_df.groupby(["resolution", "accelerator"])[
    "throughput (samples/sec)"
].idxmax()
result = viz_df.loc[grouped, viz_df.columns]
result

## Highest amount of speedup from XLA grouped by model family

Thanks to ChatGPT for the code used in this section.

### Absolute speedup

_Which model family has the highest amount of speedup from XLA for a particular resolution (say 224) and accelerator (say A100)?_

In [ ]:
# Filter rows w.r.t the resolution of 224 and A100 accelerator.
viz_df_224_a100 = viz_df.query(f"resolution == 224 and accelerator == 'a100'")

# Filter rows where xla is True (XLA enabled).
xla_enabled = viz_df_224_a100[viz_df_224_a100["xla"]]

# Filter rows where xla is False (XLA disabled).
xla_disabled = viz_df_224_a100[~viz_df_224_a100["xla"]]

# Group by 'model_family' and calculate the speedup for each model variant.
grouped = []
for model_family, group in xla_enabled.groupby("model_family"):
    for model_variant, variant_group in group.groupby("model_variant"):
        throughput_with_xla = variant_group["throughput (samples/sec)"].values[0]
        throughput_without_xla = xla_disabled[
            (xla_disabled["model_family"] == model_family)
            & (xla_disabled["model_variant"] == model_variant)
        ]["throughput (samples/sec)"].values[0]
        speedup = throughput_with_xla - throughput_without_xla
        grouped.append(
            {
                "model_family": model_family,
                "model_variant": model_variant,
                "speedup": speedup,
            }
        )

# Create a dataframe from the grouped results.
result = pd.DataFrame(grouped)

# Find the model variant with the highest speedup per model family.
max_speedup = result.groupby("model_family")["speedup"].idxmax()
result_max_speedup = result.loc[max_speedup]
result_max_speedup.sort_values(by="speedup", ascending=False)

### In terms of relative percentages

In [ ]:
# Filter rows where xla is True (XLA enabled).
xla_enabled = viz_df_224_a100.query("xla == True")

# Filter rows where xla is False (XLA disabled).
xla_disabled = viz_df_224_a100.query("xla == False")

# Group by 'model_family' and calculate the speedup for each model variant.
grouped = []
for model_family, group in xla_enabled.groupby("model_family"):
    for model_variant, variant_group in group.groupby("model_variant"):
        throughput_with_xla = variant_group["throughput (samples/sec)"].values[0]
        throughput_without_xla = xla_disabled.query(
            "model_family == @model_family and model_variant == @model_variant"
        )["throughput (samples/sec)"].values[0]
        speedup_percentage = (
            (throughput_with_xla - throughput_without_xla) / throughput_without_xla
        ) * 100
        grouped.append(
            {
                "model_family": model_family,
                "model_variant": model_variant,
                "speedup_percentage": speedup_percentage,
            }
        )

# Create a dataframe from the grouped results.
result = pd.DataFrame(grouped)

# Find the model variant with the highest speedup percentage per model family.
max_speedup = result.groupby("model_family")["speedup_percentage"].idxmax()
result_max_speedup = result.loc[max_speedup]
result_max_speedup.sort_values(by="speedup_percentage", ascending=False)